In [1]:
import boa, os
from dotenv import load_dotenv
from redux import display_erc20, lesson3

# Constants
load_dotenv(".env")
RPC_URL = f"https://arb-mainnet.g.alchemy.com/v2/{os.getenv('ARB_ALCHEMY_KEY')}"
ARBISCAN_KEY = os.getenv("ARBISCAN_KEY")
ARBISCAN_API = "https://api.arbiscan.io/api"

user = boa.env.eoa
vault_addr, collateral_token, factory = lesson3()

print(f"Loaded from lesson 3 (https://www.youtube.com/watch?v=yf5gbd4sK2c)")
print(f" • Factory: {factory.address}\n • Vault: {vault_addr}\n • VyperWIF: {collateral_token.address}")

Loaded from lesson 3 (https://www.youtube.com/watch?v=yf5gbd4sK2c)
 • Factory: 0xcaEC110C784c9DF37240a8Ce096D352A75922DeA
 • Vault: 0xC8248953429d707C6A2815653ECa89846Ffaa63b
 • VyperWIF: 0x0880cf17Bd263d3d3a5c09D2D86cCecA3CcbD97c


## Load the Llama Lend Vault and Controller from a Blueprint ABI

In [2]:
from boa.contracts.abi.abi_contract import ABIContractFactory
from boa.explorer import fetch_abi_from_etherscan


def load_from_impl(contract_addr, abi_addr, name=None):
    if name is None:
        name = contract_addr
    abi = fetch_abi_from_etherscan(abi_addr, ARBISCAN_API, ARBISCAN_KEY)
    return ABIContractFactory.from_abi_dict(abi, name=name).at(contract_addr)


vault = load_from_impl(vault_addr, factory.vault_impl(), "VyperWifVault")
controller = load_from_impl(vault.controller(), factory.controller_impl(), "VyperWifController")
amm = load_from_impl(vault.amm(), factory.amm_impl(), 'VyperWifAMM')

print(f"{vault._name} loaded: {vault.address}")
print(f"{controller._name} loaded: {controller.address}")
print(f"{amm._name} loaded: {amm.address}")

VyperWifVault loaded: 0xC8248953429d707C6A2815653ECa89846Ffaa63b
VyperWifController loaded: 0x83B85f3b08B5EE58dE9EF9604e7Eec087FCCf130
VyperWifAMM loaded: 0x9216944b568A53CF63dbF15D93f0F93F6ACb09B7


## Mint crvUSD

In [3]:
vault_crvusd_balance = 100_000 * 10 ** 18
crvusd = boa.from_etherscan(factory.STABLECOIN(), "crvUSD", ARBISCAN_API, ARBISCAN_KEY)


def bridge_mint(crvusd, user, amount):
    with boa.env.prank(crvusd.l2Gateway()):
        crvusd.bridgeMint(user, amount)


print(f"Init $crvUSD balance: {display_erc20(crvusd.balanceOf(user))}")
bridge_mint(crvusd, user, vault_crvusd_balance)
print(f"Final $crvUSD balance: {display_erc20(crvusd.balanceOf(user))}")

Init $crvUSD balance: 0
Final $crvUSD balance: 100,000


## Deposit crvUSD to the Vault Contract

In [4]:
print("Init $crvUSD balance")
print(f" • User: {display_erc20(crvusd.balanceOf(user))}")
print(f" • Controller: {display_erc20(crvusd.balanceOf(controller))}\n")

# Approve
crvusd.approve(vault, crvusd.balanceOf(user))

# Supply $crvUSD
vault.deposit(crvusd.balanceOf(user))

print("Final $crvUSD balance")
print(f" • User: {display_erc20(crvusd.balanceOf(user))}")
print(f" • Controller: {display_erc20(crvusd.balanceOf(controller))}")

Init $crvUSD balance
 • User: 100,000
 • Controller: 0

Final $crvUSD balance
 • User: 0
 • Controller: 100,000


## Create a $crvUSD loan using your VyperWIF memecoin as collateral

In [5]:
print(f"Init $VyperWIF balance \n • User: {display_erc20(collateral_token.balanceOf(user))}") 
print(f" • AMM: {display_erc20(collateral_token.balanceOf(vault.amm()))}\n")

n_bands = 4

# Approve
collateral_amount = 100_000 * 10 ** 18
borrow_amount = controller.max_borrowable(collateral_amount, n_bands)
collateral_token.approve(controller, collateral_amount)

# Borrow
controller.create_loan(collateral_amount, borrow_amount, n_bands)

print(f"Final $VyperWIF balance \n • User: {display_erc20(collateral_token.balanceOf(user))}")
print(f" • AMM: {display_erc20(collateral_token.balanceOf(vault.amm()))}")

Init $VyperWIF balance 
 • User: 100,000,000
 • AMM: 0

Final $VyperWIF balance 
 • User: 99,900,000
 • AMM: 100,000


## Create Curve Pool

In [6]:
pool_factory = boa.from_etherscan('0x98ee851a00abee0d95d08cf4ca2bdce32aeaaf7f', "Factory", ARBISCAN_API, ARBISCAN_KEY)
init_count = pool_factory.pool_count()
implementation_id = 0


pool_factory.deploy_pool('SnekWIF/crvUSD', 
                         'SnekUSD', 
                         [factory.STABLECOIN(), collateral_token],
                         implementation_id, 
                         400_000,                    # A Parameter 
                         int(0.000145 * 10 ** 18),   # Gamma
                         int(0.26 * 10 ** 8),        # Mid Fee
                         int(0.45 * 10 ** 8),        # Out Fee
                         int(0.00023 * 10 ** 18),    # Fee Gamma
                         int(0.000002 * 10 ** 18),   # Allowed Extra Profit
                         int(0.000146 * 10 ** 18),   # Adjustment Step 
                         600,                        # Moving Average Time
                         amm.price_oracle()
                        )

assert pool_factory.pool_count() > init_count

liquidity_pool_addr = pool_factory.pool_list(pool_factory.pool_count() - 1)
liquidity_pool = load_from_impl(
    liquidity_pool_addr, 
    pool_factory.pool_implementations(implementation_id), 
    'VyperWIFPool'
)

print(f"Successfully deployed {liquidity_pool.name()} to {liquidity_pool_addr}")

Successfully deployed SnekWIF/crvUSD to 0x2C7BDFb68fc4A36661E4E7A5F5F7Aa636f7B99df


## Seed Curve Pool

In [7]:
pool_crvusd_balance = int(10_000_000 * 10 ** 18)
pool_token_balance = int(amm.price_oracle() * pool_crvusd_balance / 10 ** 18)

assert collateral_token.balanceOf(user) > pool_token_balance
bridge_mint(crvusd, user, pool_crvusd_balance)

# Approve
crvusd.approve(liquidity_pool, pool_crvusd_balance)
collateral_token.approve(liquidity_pool, pool_token_balance)

# Seed
liquidity_pool.add_liquidity([pool_crvusd_balance, pool_token_balance], 0)
print(f"Deposited and received {display_erc20(liquidity_pool.balanceOf(user))} LP tokens")

Deposited and received 10,000,000 LP tokens
